In [9]:
import pandas as pd
import numpy as np
import random
import pickle

templates = [
    "list the TYPE made by DIRECTOR",
    "Present all TYPE crafted by DIRECTOR.",
    "who directed TITLE?",
    "who was the director of TITLE?",
    "which TYPE was with CAST?",
    "in which TYPE there where CAST as actors?",
    "who acted in TITLE?",
    "list the TYPE with genre GENRE",
    "which TYPE are rated RATING?",
    "which TYPE where made in COUNTRY?",
    "list the TYPE under the GENRE category",
    "which TYPE were released in YEAR?",
    "which TYPE were released in YEAR and have a duration shorter than DURATION?",
    "which TYPE last more than DURATION?",
    "which TYPE last less than DURATION?",
    "list the TYPE listed in GENRE"
]

In [10]:
df = pd.read_csv('../data/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [11]:
train_size = int(0.7 * df.shape[0])
val_size = int(0.2 * df.shape[0])

sets = {
    "train": df.iloc[:train_size,:],
    "val": df.iloc[train_size:train_size+val_size,:],
    "test": df.iloc[train_size+val_size:,:]
}

In [13]:
for set, data in sets.items():
    question_set = []
    for index, row in data.iterrows():
        
        sentence = random.choice(templates)
        entities = []

        if 'TYPE' in sentence:
            sentence = sentence.replace('TYPE', row['type'])
            entities.append([row['type'], 'TYPE'])
        if 'DIRECTOR' in sentence:
            if row['director'] != row['director']:
                continue
            valuesToReplace = row['director'].split(', ')
            if len(valuesToReplace) == 1:
                sentence = sentence.replace('DIRECTOR', valuesToReplace[0])
            else:
                sentence = sentence.replace('DIRECTOR', ', '.join(valuesToReplace[:-1])+' and '+valuesToReplace[-1])
            for value in valuesToReplace:
                entities.append([value, 'DIRECTOR'])
        if 'TITLE' in sentence:
            sentence = sentence.replace('TITLE', row['title'])
            entities.append([row['title'], 'TITLE'])
        if 'RATING' in sentence:
            sentence = sentence.replace('RATING', str(row['rating']))
            entities.append([str(row['rating']), 'RATING'])
        if 'YEAR' in sentence:
            sentence = sentence.replace('YEAR', str(row['release_year']))
            entities.append([str(row['release_year']), 'YEAR'])
        if 'DURATION' in sentence:
            sentence = sentence.replace('DURATION', str(row['duration']))
            entities.append([str(row['duration']), 'DURATION'])
        if 'CAST' in sentence:
            if row['cast'] != row['cast']:
                continue
            valuesToReplace = row['cast'].split(', ')
            if len(valuesToReplace) == 1:
                sentence = sentence.replace('CAST', valuesToReplace[0])
            else:
                sentence = sentence.replace('CAST', ', '.join(valuesToReplace[:-1])+' and '+valuesToReplace[-1])
            for value in valuesToReplace:
                entities.append([value, 'CAST'])
        if 'GENRE' in sentence:
            valuesToReplace = row['listed_in'].split(', ')
            if len(valuesToReplace) == 1:
                sentence = sentence.replace('GENRE', valuesToReplace[0])
            else:
                sentence = sentence.replace('GENRE', ', '.join(valuesToReplace[:-1])+' and '+valuesToReplace[-1])
            for value in valuesToReplace:
                entities.append([value, 'GENRE'])
        if 'COUNTRY' in sentence:
            if row['country'] != row['country']:
                continue
            valuesToReplace = row['country'].split(', ')
            if len(valuesToReplace) == 1:
                sentence = sentence.replace('COUNTRY', valuesToReplace[0])
            else:
                sentence = sentence.replace('COUNTRY', ', '.join(valuesToReplace[:-1])+' and '+valuesToReplace[-1])
            for value in valuesToReplace:
                entities.append([value, 'COUNTRY'])

        ents = []
        for element in entities:
            start = sentence.find(element[0])
            if start == -1:
                break
            end = start + len(element[0])
            ents.append([start, end, element[1]])


        question_set.append([sentence, {"entities": sorted(ents, key=lambda x: x[0])}])

    with open(f'{set}.pkl','wb') as f:
        pickle.dump(question_set, f)